Resourses
https://github.com/haoyye/End2End_GAN/tree/master
https://github.com/matusstas/cGAN/blob/main/cgan.ipynb

 Check GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Thu Feb 22 00:25:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX230         WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   43C    P0              N/A / ERR! |      0MiB /  2048MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Import dependencies

In [8]:
# wandb (weights and biases) -> tracking tool
!pip install -q wandb 

In [3]:
import numpy as np
import tensorflow as tf

#To datasets -> import tensorflow_datasets as tfds

# For models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, Embedding, Input, Concatenate #, Conv2DTranspose

# Callbacks
# Visit this -> https://kvirajdatt.medium.com/essential-tensorflow-and-keras-callbacks-for-your-neural-networks-54539244db39#:~:text=Some%20advantages%20of%20using%20these,of%20epochs%20for%20your%20training
from tensorflow.keras.callbacks import Callback

# For streamlining ML workflow
import wandb
from wandb.keras import WandbCallback

# For model memory
from tensorflow.keras import backend as k
import humanize

# For visualisation
from matplotlib import pyplot as plt


Set data

In [83]:
n = 7
k = 4

Input_Shape = (2*n,1)#(n*2,1)   # Shape of the conditional input (this is normalized)
noise_std = 1           # Standerd deviation for the noise to be added to the conditional values for better generalization (change this)

"""---- Import dataset here ----"""

"""--------"""

# latent_vector_dim = 100 # For the Generator input
batch_size = 320

#n_batches = len(data//batch_size)

N_epochs = 10

# Name of the project for Weights and Biases platform
WB_project = "CGAN"

# Entity (login) for Weights and Biases platform
WB_entity = "matusstas" # shearch this


In [87]:
"""Making data - Get real data"""

# Here for every 320 messages, 1/4 of them (90) are pilot symbols . every message has 2*n values (n complex values)
num_frames = 100
dummy_data = np.random.uniform(-1,1,num_frames*batch_size*2*n)

n_batches = len(dummy_data//batch_size)

def nakagami_channel(iq_samples, m, omega, snr_db):
    # Generate Nakagami fading coefficients
    gamma = np.random.gamma(m, scale=1/m, size=iq_samples.shape)

    # Apply fading to in-phase and quadrature components
    faded_iq = iq_samples * np.sqrt(gamma)

    # Simulate AWGN
    noise_var = 10**(-snr_db/10) / (2 * m)  # Convert SNR dB to noise variance
    noise = np.random.normal(scale=np.sqrt(noise_var), size=iq_samples.shape) #+ 1j * np.random.normal(scale=np.sqrt(noise_var), size=iq_samples.shape)

    # Add noise to faded signal
    return faded_iq + noise

i_val = int(num_frames*batch_size/4*2*n)
x = np.zeros(i_val*3)
y = np.zeros(i_val*3)
pilot_x = np.zeros(i_val)
pilot_y = np.zeros(i_val)

m_list = [0.5,1,1.5]
snr_list = [1,3,6] 

i_val= int(batch_size/4*2*n)
for i in range(num_frames):
    
    left_i_xy = 3*i_val*i
    right_i_xy = left_i_xy + 3*i_val
    left_i_p = i*i_val
    right_i_p = left_i_p + i_val


    m = np.random.choice(m_list)
    omega = 2*np.pi
    snr = np.random.choice(snr_list)

    x_hat = dummy_data[i*batch_size*2*n:i*batch_size*2*n+batch_size*2*n]
    x[left_i_xy:right_i_xy] = x_hat[0:3*i_val]
    pilot_x[left_i_p:right_i_p] = x_hat[3*i_val:batch_size*2*n]
    y_hat = nakagami_channel(x_hat,m,omega,snr)
    y[left_i_xy:right_i_xy] = y_hat[0:3*i_val]
    pilot_y[left_i_p:right_i_p] = y_hat[3*i_val:batch_size*2*n]

    # for every 90*2*n values in pilot, there are 270*2*n values for normal signals
    

In [93]:
print(x.shape)
print(y.shape)
print(pilot_x.shape)
print(pilot_y.shape)

(336000,)
(336000,)
(112000,)
(112000,)


Calculate channel function

In [127]:

def real_to_complex(a):
    a= np.reshape(a,(len(a)//2,2))
    a= np.vectorize(complex)(np.transpose(a[:,0]),np.transpose(a[:,1]))
    print(a)
    return a

h = np.matmul(real_to_complex(x).conj().T,real_to_complex(y))/np.matmul(real_to_complex(x).conj().T,real_to_complex(x))
print(h)


[ 0.11251814+0.89331898j  0.87539909-0.67693898j  0.63174895+0.64181104j
 ... -0.06901767+0.21416178j  0.63896359-0.40965382j
 -0.92664883-0.35104461j]
[-0.03893125-0.38876497j  0.42148561-1.41094349j  0.681181  -1.03244978j
 ... -0.55325849+0.08069552j -0.16755595-0.43548219j
 -1.05568085+0.50445823j]
[ 0.11251814+0.89331898j  0.87539909-0.67693898j  0.63174895+0.64181104j
 ... -0.06901767+0.21416178j  0.63896359-0.40965382j
 -0.92664883-0.35104461j]
[ 0.11251814+0.89331898j  0.87539909-0.67693898j  0.63174895+0.64181104j
 ... -0.06901767+0.21416178j  0.63896359-0.40965382j
 -0.92664883-0.35104461j]
(0.8718785362900613+0.0025636499707518976j)


In [131]:
a = np.array([[1,2],[3,4],[7,8]])
print(np.matmul(a,a.T))

[[  5  11  23]
 [ 11  25  53]
 [ 23  53 113]]


### Build Models

Generator

In [ ]:
def build_generator():
    